In [1]:
import tensorflow
from __future__ import division, print_function, unicode_literals

# Introduction
For our final project we are going to be using the CIFAR10 dataset. This dataset has 60000 color images that are divided into ten different classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. These images are 32x32 pixels and 50000 of them are dedicated as training images and 10,000 of them are for testing. Each of the classes has 5,000 images. The labels are numerical numbers 0-9 that represent the different classes.

# Plan
Our plan is to create a Convolutional Neural Network that can get the highest accuracy on the test set. Some of our concerns are if there are too many images for the computing power we have currently. There are many paid options we can utilize, but computing time is a concern with us.

First, we are planning to try and putting our data into the regular mnist CNN we have been using in class. From there, we will make the necessary changes. Some of the things we want to look into is data augmentation and how creating new data objects from the images we already have would help training.

## Getting the data

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

dict = []
files = ["cifar-10-batches-py/data_batch_1", "cifar-10-batches-py/data_batch_2", "cifar-10-batches-py/data_batch_3", "cifar-10-batches-py/data_batch_4", "cifar-10-batches-py/data_batch_5"]
for i in []